# Summarizing (w/ GPT-2)

## Packages

In [32]:
from utils.json_utils import read_json, write_json
from datasets import Dataset
import pandas as pd

## Tokenizing

In [33]:
sentences_w_subjects_tokenized = read_json("9_non_lemmatized_tokenized_sentences_black_clover.json")
sentences_w_subjects_tokenized

[{'subjects': ['Yuno'],
  'tokens': ['a',
   'priest',
   'takes',
   'two',
   'babies',
   'abandoned',
   'outside',
   'his',
   'church',
   'inside',
   'and',
   'discovers',
   'two',
   'babies',
   'abandoned',
   'outside',
   'his',
   'church',
   'names',
   'to',
   'be',
   'Yuno',
   'and',
   'Asta',
   '.']},
 {'subjects': ['Asta'],
  'tokens': ['a',
   'priest',
   'takes',
   'two',
   'babies',
   'abandoned',
   'outside',
   'his',
   'church',
   'inside',
   'and',
   'discovers',
   'two',
   'babies',
   'abandoned',
   'outside',
   'his',
   'church',
   'names',
   'to',
   'be',
   'Yuno',
   'and',
   'Asta',
   '.']},
 {'subjects': ['Lily'],
  'tokens': ['Fifteen',
   'years',
   'later',
   ',',
   'Asta',
   'proposes',
   'to',
   'Sister',
   'Lily',
   ',',
   'who',
   'refuses',
   'repeatedly',
   '.']},
 {'subjects': ['Asta'],
  'tokens': ['Fifteen',
   'years',
   'later',
   ',',
   'Asta',
   'proposes',
   'to',
   'Sister',
   'Lily',
   

In [34]:
sentences_w_subjects_tokenized = [
    {
        "subjects" : sentence["subjects"],
        "tokens"  : " ".join(sentence["tokens"])
    } 
    for sentence in sentences_w_subjects_tokenized
]
sentences_w_subjects_tokenized

[{'subjects': ['Yuno'],
  'tokens': 'a priest takes two babies abandoned outside his church inside and discovers two babies abandoned outside his church names to be Yuno and Asta .'},
 {'subjects': ['Asta'],
  'tokens': 'a priest takes two babies abandoned outside his church inside and discovers two babies abandoned outside his church names to be Yuno and Asta .'},
 {'subjects': ['Lily'],
  'tokens': 'Fifteen years later , Asta proposes to Sister Lily , who refuses repeatedly .'},
 {'subjects': ['Asta'],
  'tokens': 'Fifteen years later , Asta proposes to Sister Lily , who refuses repeatedly .'},
 {'subjects': ['Yuno'],
  'tokens': 'Yuno and the other orphans criticize Asta and point out Yuno lack of magic .'},
 {'subjects': ['Asta'],
  'tokens': 'Yuno and the other orphans criticize Asta and point out Yuno lack of magic .'},
 {'subjects': ['Yuno'],
  'tokens': 'Asta tries to show off Asta skills , but Yuno outshines Asta with Asta magic .'},
 {'subjects': ['Asta'],
  'tokens': 'Asta t

In [35]:
max_length = 0
for sentence in sentences_w_subjects_tokenized:
    if len(sentence["tokens"]) > max_length:
        max_length = len(sentence["tokens"])
max_length

610

In [36]:
varied_set_adjectives = read_json("14_varied_set_adjectives_definitions.json")
#varied_set_adjectives

In [37]:
training_sents = list(map(lambda x : x["tokens"], sentences_w_subjects_tokenized))
#training_sents

## Transformer

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

In [39]:
model_checkpoint = "distilgpt2"

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/pauli/.cache/huggingface/hub/models--distilgpt2/snapshots/f241065e938b44ac52db2c5de82c8bd2fafc76d0/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_t

In [41]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [42]:
tokenize_fn = lambda doc : tokenizer(
        doc,
        truncation=True,
        max_length=1024
    )

tokenized_dataset = list(map(tokenize_fn, training_sents))

In [43]:
tokenized_dataset = pd.DataFrame(tokenized_dataset)
tokenized_dataset = Dataset.from_pandas(tokenized_dataset)

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, pad_token_id=tokenizer.eos_token_id)

loading configuration file config.json from cache at /home/pauli/.cache/huggingface/hub/models--distilgpt2/snapshots/f241065e938b44ac52db2c5de82c8bd2fafc76d0/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj"

In [45]:
device = f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"

In [46]:
training_args = TrainingArguments(
    f"{model_checkpoint}-finetuned-sents-coref-black-clover",
    num_train_epochs=5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,    
    learning_rate=2e-5,
    weight_decay=0.01,
    do_eval=True, # eval en validation set
    evaluation_strategy="steps", # eval en validation set
    eval_steps=100,
    save_steps=100, # checkpoint model every 500 steps
    logging_dir='./logs', # logging
    logging_strategy="steps",
    logging_steps=1,
    fp16=False, # float16 en training (only on CUDA)
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset, #.select(range(0, 128)),
    eval_dataset=tokenized_dataset, #.select(range(0, 128)),
)

In [48]:
train_output = trainer.train()

/home/pauli/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5989
  Num Epochs = 2
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1198
  0%|          | 1/1198 [00:02<43:35,  2.19s/it]

{'loss': 5.335, 'learning_rate': 1.9983305509181973e-05, 'epoch': 0.0}


  0%|          | 2/1198 [00:04<40:25,  2.03s/it]

{'loss': 5.9918, 'learning_rate': 1.996661101836394e-05, 'epoch': 0.0}


  0%|          | 3/1198 [00:05<32:57,  1.65s/it]

...

{'loss': 2.8243, 'learning_rate': 3.33889816360601e-08, 'epoch': 2.0}


100%|█████████▉| 1197/1198 [1:13:14<00:02,  2.17s/it]

{'loss': 3.1005, 'learning_rate': 1.669449081803005e-08, 'epoch': 2.0}


100%|██████████| 1198/1198 [1:13:16<00:00,  1.91s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1198/1198 [1:13:16<00:00,  3.67s/it]

{'loss': 3.4882, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 4396.1106, 'train_samples_per_second': 2.725, 'train_steps_per_second': 0.273, 'train_loss': 3.5160807968181045, 'epoch': 2.0}


In [49]:
def generate(
    prompt=None, max_length=1024, max_new_tokens=20, greedy=True, model=model, tokenizer=tokenizer, device=device
):
    """None stands for beggining of sequence.
    NOTE si bien parece que GPT2 puede generar a partir de BOS token, la 
    documentacion es poco clara. Ademas hicimos nuestro finetuning sin BOS token.
    Entonces solo vamos a usar la funcion pasandole un contexto.

    Ver:
    https://github.com/huggingface/transformers/issues/3311#issuecomment-601264426
    https://github.com/openai/gpt-2/blob/a74da5d99abaaba920de8131d64da2862a8f213b/src/generate_unconditional_samples.py#L60
    """
    do_sample = False if greedy else True
    # model.eval() to set dropout and batch normalization layers to evaluation 
    # mode before running inference
    if prompt:
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        model.eval()
        outputs = model.generate(input_ids, do_sample=do_sample, max_new_tokens=max_new_tokens)
    else:
        model.eval()
        outputs = model.generate(do_sample=do_sample, max_new_tokens=max_new_tokens)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

## Summarize

In [50]:
generate("Microsoft's CEO is ", greedy=False)

["Microsoft's CEO is iced to ask Yuno how Asta can not handle. No, Yuno responds that Asta"]

In [51]:
import re

In [52]:
characters = read_json('3_characters_black_clover.json')

In [53]:
non_greedy_generated_descriptions = []

for subject in characters:
    generated_doc = generate(subject + " can be described as ", max_length=1024, greedy=False)
    generated_desc = re.sub(r"\s+", " ", (re.split(r"[.;:!?]", generated_doc[0]))[0])
    non_greedy_generated_descriptions.append(generated_desc)

In [54]:
greedy_generated_descriptions = []
for subject in characters:
    generated_doc = generate(subject + " can be described as ", max_length=1024, greedy=True)
    generated_desc = re.sub(r"\s+", " ", (re.split(r"[.;:!?]", generated_doc[0]))[0])
    greedy_generated_descriptions.append(generated_desc)

In [55]:
re.split("\.", "hol.")

['hol', '']

In [56]:
non_greedy_generated_descriptions

['Acier can be described as ills, confusion, and chaos, though Vermillion does not fully understand the situation and tries to explain',
 "Noelle can be described as 's sister and Yami Sukehiro goes looking at everyone before they can speak",
 'Damnatio can be described as icky and arrogant, while Silva does have another spell of attack to help Yuno, and while Yun',
 'Nozel can be described as ㄓ is impressed with what Julius has learned',
 'Solid can be described as ierry brother',
 'Mereleona can be described as iced black, which is why Lotte asks Noelle to be more like Vanessa',
 "Fuegoleon can be described as Â's kind of devil, but Vermillion says that Fuegoleon, if not the",
 'Leopold can be described as iced magic by Gauche Vermillion and Mimosa Vermillion',
 'Kirsch can be described as 〉 Magic Magic Knight, having fought back from a distance and fighting hard',
 'Mimosa can be described as iced and is not familiar with any magic, but Mimosa tells Mimosa that the devil that magic'

In [57]:
greedy_generated_descriptions

['Acier can be described as iced up and is able to use Asta Magic to heal Asta',
 'Noelle can be described as ichai, but Noelle is not',
 'Damnatio can be described as iced up and a bit of a bit of a bit of a bit of a bit of a bit',
 'Nozel can be described as ichai, but Nozel can be described as being a little more powerful than the others',
 "Solid can be described as ichai's magic, but the Magic Knights are not able to use it",
 'Mereleona can be described as iced up and a bit of a bit of a bit of a bit of a bit of a bit',
 'Fuegoleon can be described as ichai, but Gauche Adlai is not',
 'Leopold can be described as iced up and is not impressed by the way the elves are treated',
 'Kirsch can be described as ichai, but Klaus is not the same person',
 'Mimosa can be described as iced up and is not impressed by the way Mimosa is treated',
 'Gauche can be described as iced up and a bit of a bit of a bit of a bit of a bit of a bit',
 'Marie can be described as ichai, but Yuno is not the 

In [58]:
adjectives = read_json("14_varied_set_adjectives.json")
import numpy as np

In [59]:
perplex_adjs = dict()
for character in characters:
    perplex_adjs[character] = dict()
    for adj in adjectives:
        sent = character + " is " + adj
        tokenized_sent = tokenizer(sent)
        tokenized_sent = Dataset.from_list([tokenized_sent])
        perplexity = np.exp(trainer.evaluate(tokenized_sent)["eval_loss"])
        perplex_adjs[character][adj] = perplexity

***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 416.47it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1227.12it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1377.89it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1342.61it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1372.03it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1430.04it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1334.49it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1454.84it/s]
***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|

In [60]:
write_json(perplex_adjs, "25_perplex_adjs.json")

In [61]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5989
  Batch size = 10
100%|██████████| 599/599 [03:31<00:00,  2.84it/s]


{'eval_loss': 3.0445849895477295,
 'eval_runtime': 211.49,
 'eval_samples_per_second': 28.318,
 'eval_steps_per_second': 2.832,
 'epoch': 2.0}

In [62]:
sent = "Asta is good."

In [63]:
tokenized_sent = tokenizer(sent)
tokenized_sent

{'input_ids': [33751, 64, 318, 922, 13], 'attention_mask': [1, 1, 1, 1, 1]}

In [64]:
tokenized_sent = Dataset.from_list([tokenized_sent])
tokenized_sent

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1
})

In [65]:
trainer.evaluate(tokenized_sent)

***** Running Evaluation *****
  Num examples = 1
  Batch size = 10
100%|██████████| 1/1 [00:00<00:00, 1402.31it/s]


{'eval_loss': 5.335954189300537,
 'eval_runtime': 0.0428,
 'eval_samples_per_second': 23.355,
 'eval_steps_per_second': 23.355,
 'epoch': 2.0}

In [73]:
perplex_adjs = read_json("25_perplex_adjs.json")
sorted_perplex_adjs = sorted(perplex_adjs["Asta"].items(), key=lambda kv: kv[1])
print(sorted_perplex_adjs)

[('boisterous', 17.0191209357947), ('drowsy', 26.590678761831732), ('crestfallen', 28.281527762959684), ('flamboyant', 35.237711598410165), ('kind-hearted', 36.77458200069556), ('petulant', 40.65252537304482), ('jovial', 44.449426271374364), ('hot-headed', 49.92388635801326), ('fastidious', 59.57343192044881), ('level-headed', 59.82516384496433), ('warm-hearted', 78.46620616727674), ('easy-going', 95.84311820564454), ('cantankerous', 98.20888512266716), ('zealous', 124.09563908584191), ('tetchy', 125.08175596903227), ('reclusive', 127.09592202380453), ('nosy', 135.10961944739532), ('absent-minded', 137.07940547825945), ('shrewd', 146.23336601310876), ('lanky', 146.66479003469308), ('enthusiastic', 147.54405316629595), ('fervent', 147.70152012263088), ('funny', 149.2224771838704), ('demure', 149.24887795798426), ('irascible', 150.3248034352396), ('generous', 157.85426132708375), ('haughty', 160.61895252611447), ('meddlesome', 170.4104297190763), ('restless', 174.79098838842663), ('unsta

In [74]:
perplex_adjs = read_json("25_perplex_adjs.json")
sorted_perplex_adjs = sorted(perplex_adjs["Noelle"].items(), key=lambda kv: kv[1])
print(sorted_perplex_adjs)

[('boisterous', 25.820727203266387), ('crestfallen', 39.662667702230316), ('drowsy', 42.83037075265757), ('flamboyant', 46.16444095867613), ('kind-hearted', 56.46289185514815), ('hot-headed', 60.04943344188203), ('petulant', 64.09216908680305), ('jovial', 67.32581236780196), ('level-headed', 75.70633619804553), ('fastidious', 88.06187932511324), ('warm-hearted', 95.0384203653458), ('easy-going', 108.12599389341224), ('tetchy', 140.66411609655125), ('meddlesome', 152.19501574633685), ('reclusive', 162.52591705756493), ('cantankerous', 176.1895294452467), ('haughty', 179.25688436722), ('shrewd', 225.44302252621392), ('fervent', 232.81928378140853), ('morose', 234.32642149745485), ('nosy', 238.75424996927862), ('demure', 240.45569184519064), ('zealous', 258.902689354551), ('lanky', 264.8816804618698), ('funny', 268.6998689556166), ('enthusiastic', 270.45629953968256), ('irascible', 276.87157012136834), ('fussy', 298.62488854597433), ('generous', 300.38823083619246), ('absent-minded', 313.

In [76]:
perplex_adjs = read_json("25_perplex_adjs.json")
sorted_perplex_adjs = sorted(perplex_adjs["Zora"].items(), key=lambda kv: kv[1])
print(sorted_perplex_adjs)

[('boisterous', 46.789721255850345), ('crestfallen', 72.27494477478764), ('flamboyant', 79.70559367457724), ('drowsy', 89.54450083769001), ('kind-hearted', 100.14836501972783), ('hot-headed', 122.64836052374513), ('petulant', 135.36627555506183), ('jovial', 135.59677766952404), ('level-headed', 172.77084049077823), ('fastidious', 210.088212904695), ('warm-hearted', 220.7627350364559), ('easy-going', 283.8335313074117), ('tetchy', 304.3614399532382), ('absent-minded', 321.0852233245175), ('cantankerous', 332.049806405777), ('irascible', 372.12665837203366), ('haughty', 398.8711374859489), ('zealous', 405.31842961918557), ('reclusive', 438.20022202570925), ('shrewd', 482.43415826071237), ('nosy', 502.8050125728328), ('fervent', 518.9983338812164), ('meddlesome', 538.7398120361822), ('demure', 553.0844762585295), ('lanky', 563.0885881518657), ('morose', 633.0196358451585), ('fussy', 767.0473711748937), ('grouchy', 796.1932997448677), ('funny', 836.9329481300549), ('enthusiastic', 840.8069

In [77]:
perplex_adjs = read_json("25_perplex_adjs.json")
sorted_perplex_adjs = sorted(perplex_adjs["Lily"].items(), key=lambda kv: kv[1])
print(sorted_perplex_adjs)

[('boisterous', 101.9155525441156), ('flamboyant', 124.55122098484637), ('crestfallen', 161.95204948710966), ('drowsy', 167.4863845114523), ('kind-hearted', 205.13836288749025), ('jovial', 249.94030630299503), ('hot-headed', 285.45930068519664), ('level-headed', 298.744382223946), ('petulant', 300.28912768860766), ('warm-hearted', 414.91405545876194), ('tetchy', 469.44549499765725), ('fastidious', 501.5354312080621), ('cantankerous', 540.7733373372963), ('easy-going', 556.0435161346699), ('meddlesome', 714.258339298417), ('irascible', 734.5099826312157), ('absent-minded', 780.5441185315331), ('reclusive', 881.578001990995), ('haughty', 909.0792636616103), ('lanky', 1078.8456108954551), ('shrewd', 1103.6889729872748), ('zealous', 1129.932243730732), ('fervent', 1247.6848707261533), ('morose', 1254.1680867560297), ('demure', 1269.5092274749225), ('nosy', 1515.228213341892), ('grouchy', 1521.765883798417), ('fussy', 1642.9555316071687), ('generous', 2423.367635932611), ('funny', 2539.9641

In [71]:
print(perplex_adjs["Asta"]["absent-minded"])
print(perplex_adjs["Asta"]["zealous"])

137.07940547825945
124.09563908584191
